# ToO Candidate Processing for DESITrIP

In [143]:
import pandas as pd
from glob import glob
from astropy.time import Time

from astropy.table import Table, Column
from desispec.io import read_spectra, write_spectra
from desispec.coaddition import coadd_cameras
from tqdm.notebook import tqdm
import os
import numpy as np

In [144]:
# Assemble initial table
too_files = glob(f"/global/homes/e/emmayu/desi_too_matched*.csv")
too_tab = []
for file in too_files:
    # Read each file into a dataframe
    df = pd.read_csv(file)
    too_tab.append(df)
too_tab = pd.concat(too_tab, ignore_index=True).dropna()
too_tab.columns

Index(['RA', 'DEC', 'PMRA', 'PMDEC', 'REF_EPOCH', 'CHECKER', 'TOO_TYPE',
       'TOO_PRIO', 'OCLAYER', 'MJD_BEGIN', 'MJD_END', 'TOOID', 'desi_ra',
       'desi_dec', 'desi_spectype', 'desi_targetid', 'desi_obs_date',
       'desi_deltachi2', 'desi_tiled', 'desi_petalid'],
      dtype='object')

In [145]:
# Select time matches
def to_mjd_start(date):
    date = str(int(date))
    time_obj = Time.strptime(date, '%Y%m%d')
    return time_obj.mjd + 0.5

def to_mjd_end(date):
    date = str(int(date))
    time_obj = Time.strptime(date, '%Y%m%d')
    return time_obj.mjd + 1.5
    
too_tab['desi_mjd_start'] = too_tab['desi_obs_date'].apply(to_mjd_start)
too_tab['desi_mjd_end'] = too_tab['desi_obs_date'].apply(to_mjd_end)
filtered_too = too_tab.loc[
    (too_tab['desi_mjd_start'] >= too_tab['MJD_BEGIN']) &
    (too_tab['desi_mjd_end'] <= too_tab['MJD_END'])
]

In [148]:

filtered_too = filtered_too.dropna().drop_duplicates(subset=['desi_targetid', 'desi_obs_date']).rename(columns={'desi_tiled':'desi_tileid'})
# Columns to convert
columns_to_convert = ['desi_targetid', 'desi_obs_date', 'desi_petalid', 'desi_tileid']

# Convert columns to int
filtered_too[columns_to_convert] = filtered_too[columns_to_convert].astype(int)

# Save final selection as a pkl file
filtered_too.to_pickle('/global/homes/e/emmayu/transients_cmuS24/ToO_classification/too_transients.pkl')


In [150]:
filtered_too = pd.read_pickle('/global/homes/e/emmayu/transients_cmuS24/ToO_classification/too_transients.pkl')